In [ ]:
Name: UNEEB ULLAH      SP22-Bcs-120
      KHIZAR LODHI     Sp22-Bcs-081
LAB FINAL TEXT DETECTION.

In [ ]:
%%writefile pattern_detection_cuda.cu
#include <iostream>
#include <string>
#include <cuda_runtime.h>
#include <vector>
#include <cstring>

using namespace std;

__global__ void findWordKernel(const char* text, const char* word, int textLen, int wordLen,
                               int* positions, int* count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx + wordLen > textLen) return;

    bool match = true;
    for (int i = 0; i < wordLen; i++) {
        if (text[idx + i] != word[i]) {
            match = false;
            break;
        }
    }

    if (match) {
        int pos = atomicAdd(count, 1);
        positions[pos] = idx;
    }
}

void Past_Text() {
    cin.ignore(numeric_limits<streamsize>::max(), '\n'); // clear input buffer

    string sent, word;

    cout << "Paste Your Text Here:\n";
    getline(cin, sent);

    cout << "\nEnter Word to Find:\n";
    getline(cin, word);

    int textLen = sent.length();
    int wordLen = word.length();

    if(textLen == 0 || wordLen == 0){
        cout << "Text or word cannot be empty!\n";
        return;
    }

    char* h_text = new char[textLen];
    char* h_word = new char[wordLen];

    memcpy(h_text, sent.c_str(), textLen);
    memcpy(h_word, word.c_str(), wordLen);

    char *d_text, *d_word;
    int *d_positions, *d_count;
    cudaMalloc(&d_text, textLen * sizeof(char));
    cudaMalloc(&d_word, wordLen * sizeof(char));
    cudaMalloc(&d_positions, textLen * sizeof(int));
    cudaMalloc(&d_count, sizeof(int));
    cudaMemset(d_count, 0, sizeof(int));

    cudaMemcpy(d_text, h_text, textLen * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_word, h_word, wordLen * sizeof(char), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks = (textLen + threads - 1) / threads;

    findWordKernel<<<blocks, threads>>>(d_text, d_word, textLen, wordLen, d_positions, d_count);
    cudaDeviceSynchronize();

    int h_count;
    cudaMemcpy(&h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

    int* h_positions = new int[h_count];
    cudaMemcpy(h_positions, d_positions, h_count * sizeof(int), cudaMemcpyDeviceToHost);

    cout << "\n========== RESULT ==========\n";
    if (h_count > 0) {
        cout << "WORD FOUND! Total occurrences: " << h_count << endl;
        for (int i = 0; i < h_count; i++) {
            cout << "Word: '" << word << "' at index: " << h_positions[i] << endl;
        }
    } else {
        cout << "WORD NOT FOUND!" << endl;
    }
    cout << "============================\n";

    delete[] h_text;
    delete[] h_word;
    delete[] h_positions;
    cudaFree(d_text);
    cudaFree(d_word);
    cudaFree(d_positions);
    cudaFree(d_count);

    cout << "Press Enter to continue...";
    cin.ignore(numeric_limits<streamsize>::max(), '\n');
}


int main() {
    int cmd;
    while (true) {
        cout << "\n1- Paste Your Text\n";
        cout << "6- About\n";
        cout << "7- Exit\n";
        cout << "Enter choice: ";
        cin >> cmd;

        switch (cmd) {
            case 1: Past_Text(); break;
            case 7: exit(0);
            default: cout << "Option not implemented or invalid, try again.\n"; break;
        }
    }
    return 0;
}


Overwriting pattern_detection_cuda.cu


In [ ]:
!nvcc pattern_detection_cuda.cu -o pattern_detection_cuda -arch=sm_75
!./pattern_detection_cuda


1- Paste Your Text
6- About
7- Exit
Enter choice: 1
Paste Your Text Here:
bomb bomb 

Enter Word to Find:
killer

========== RESULT ==========
WORD NOT FOUND!
Press Enter to continue...

1- Paste Your Text
6- About
7- Exit
Enter choice: 1
Paste Your Text Here:
bomb bomb bomb bomb

Enter Word to Find:
bomb

========== RESULT ==========
WORD FOUND! Total occurrences: 4
Word: 'bomb' at index: 0
Word: 'bomb' at index: 5
Word: 'bomb' at index: 10
Word: 'bomb' at index: 15
Press Enter to continue...

1- Paste Your Text
6- About
7- Exit
Enter choice: 7


In [1]:
import cupy as cp
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display, clear_output


uploader = widgets.FileUpload(accept='.txt', multiple=False)
display(widgets.HTML("<b>Upload a text file:</b>"))
display(uploader)

word_input = widgets.Text(
    value='',
    placeholder='Enter Word ( Detection )',
    description='Word:',
)
display(word_input)

output = widgets.Output()
display(output)

uploaded_text = None

def on_file_upload(change):
    global uploaded_text
    if len(uploader.value) == 0:
        return
    uploaded_file = list(uploader.value.values())[0]
    content = uploaded_file['content']
    uploaded_text = content.decode('utf-8')
    with output:
        clear_output()
        print(f"File '{uploaded_file['metadata']['name']}' uploaded successfully!")

uploader.observe(on_file_upload, names='value')

def on_word_enter(change):
    global uploaded_text
    if uploaded_text is None:
        with output:
            clear_output()
            print("Please upload a text file first!")
        return

    word = word_input.value.strip()
    if not word:
        with output:
            clear_output()
            print("Please enter a word to search!")
        return


    text_bytes = cp.array(bytearray(uploaded_text, 'utf-8'))
    word_bytes = cp.array(bytearray(word, 'utf-8'))

    text_len = text_bytes.shape[0]
    word_len = word_bytes.shape[0]

    index = -1

    for i in range(text_len - word_len + 1):
        if cp.all(text_bytes[i:i+word_len] == word_bytes):
            index = i
            break

    with output:
        clear_output()
        if index != -1:
            print(f"Word '{word}' found at index: {index}")
        else:
            print(f"Word '{word}' not found in the file.")

word_input.on_submit(on_word_enter)


HTML(value='<b>Upload a text file:</b>')

FileUpload(value={}, accept='.txt', description='Upload')

Text(value='', description='Word:', placeholder='Enter Word ( Detection )')

Output()